# Welcome back to the second DataJoint webinar!

This webinar builds on the mouse electrophysiology pipeline we started building in the first webinar. By the end of this webinar you will have learnt how to...

* ...import neuron activity data from files into an `Imported` table
* ...compute various statistics for each neuron using a `Computed` table
* ...store computation parameters in a `Lookup` table
* ...perform spike detection using another `Computed` table
* ...trigger automatic computations for all missing entries in the aforementioned tables using the `populate` method

First of all let us import DataJoint again:

In [ ]:
import datajoint as dj

We will also need NumPy to perform the computations and Matplotlib to visualize the results:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

Furthermore we need the previously defined `Mouse` and `Session` tables or more specifically their corresponding Python classes. These are conveniently provided in the `session02` module of the `webinar` package from which we can import them. We also want to import the `schema` object to be able to define additional tables:

In [ ]:
from webinar.session02 import Mouse, Session, schema

Let's take a look at the ERD of the schema to remind ourselves what we have accomplished in the previous webinar:

In [ ]:
dj.Diagram(schema)

In [ ]:
Mouse()

In [ ]:
Session()

These tables live in a new schema different from the one you used in the first webinar and they are already prefilled with data to ensure that we are all on the same page.

## Exercise: Recapping the first webinar

Insert a female mouse with ID 42 that was born on the 12th of May 2018 into the `Mouse` table:

Fetch the start and end times of the session performed on the mouse with ID 0 on the 15th of May 2017 from the `Session` table:

Fetch the sex of all mice that had sessions performed on them as a list of dictionaries from the `Mouse` table:

Delete the mouse with ID 42 from the `Mouse` table:

## Importing data from files

Recall from the project description
> * In each experimental session you record electrical activity from a single **neuron** and you use recording equipment that produces separate data files for each recorded neuron.

Because we record from a single neuron in each session and the data from each recorded neuron is stored in a single file we should have a single file per recording session.

You can find these files in the `../data` directory. They contain saved Numpy arrays as indicated by their `.npy` extension and their names contain the mouse ID and the recording session date in the format `data_{mouse_id}_{session_date}.npy`. For example the file with name `data_100_2017-05-25.npy` contains data from the recording session perfomed on the 25th of May 2017 on the mouse with ID 100.

### Looking at the data

Let's take a quick look at the contents of the data files to get a better understandin of what we will be working with.

First we fetch the primary keys of all recording sessions by passing the special `"KEY"` argument to the `fetch` method:

In [ ]:
keys = Session.fetch("KEY")
keys

Remember that each key in the resulting list can be used to uniquely identify a particular recording session:

In [ ]:
# ENTER YOUR CODE! - restrict the session table with a single key of your choosing

For now let us select the first key to work with:

In [ ]:
key = keys[0]
key

Next we want to create the name of the file corresponding to the selected key. Remember that we need the `mouse_id` and the `session_date` to create the names of our data files and conveniently precisely this information is contained in the key we just selected.

In [ ]:
filename = "../data/data_{mouse_id}_{session_date}.npy".format(**key)
filename

Here we used Python's dictionary unpacking and the `format` method to plug the correct values into our filename convention.

Finally let's load the file:

In [ ]:
data = np.load(filename)

Look at its content...

In [ ]:
data

...and check the shape of the data:

In [ ]:
data.shape

So the file contains a NumPy array of with 1000 entries. This represents a (simulated) recording of raw electric activity from a single neuron over 1000 time bins.

### Defining the Neuron table

Now we would like to have all these recorded neurons represented in the pipeline and store their activity alongside them. This will allow us to perform computations in the pipeline using the stored data.

Remember that we record from a single neuron in each session and therefore each neuron can be uniquely identified by a particular session. This in turn means that our `Neuron` table should depend on the already existing `Session` table. Furthermore we want to include a non-primary key attribute which will contain the neuron's recoreded activity imported from the respective data file. 

Let's define the `Neuron` table:

In [ ]:
@schema
class Neuron(dj.Imported):
    definition = """
    -> Session
    ---
    activity: longblob  # electric neuron activity
    """

Note that we used the `longblob` datatype to store the activity of our neurons. This datatype can be used to store arbitrary numeric arrays like for example our NumPy arrays which will be imported from the data file corresponding to each neuron. Furthermore we used the `dj.Imported` class as the base class for our `Neuron` table instead of the previously used `dj.Manual` class. This indicates that the contents of the table will depend on data imported from external files.

Let's check the state of our pipeline after the creation of the `Neuron` table:

In [ ]:
# ENTER YOUR CODE! - plot the ERD of the schema

As you can see our new `Neuron` table is represented by a blue ellipse, nicely distinguishing itself from the previously defined manual tables represented by green rectangles.

### DataJoint data tiers

Each table in DataJoint belongs to a certain data tier. The data tier of a particular table indicates the **nature of the data it contains and the source of that data**. So far we have been introduced to two data tiers: Manual and Imported. We will encounter the two other major data tiers shortly.

DataJoint tables belonging to the Manual data tier, or simply **Manual tables**, indicate that their contents are **manually** entered by either the experimenter or some system external to DataJoint and their contents **do not depend on other tables or external files**. The Manual data tier is the most basic data tier you will encounter and especially tables at the beginning of a pipeline will belong to that tier.

**Imported tables** on the other hand are understood to import their contents from external files and they come equipped with functionality to perform this importing process automatically, as we will see momentarily. 

### Importing data into the Imported table

Rather than manually adding entries to the `Neuron` table using the `insert1` or `insert` methods, we will make use of the `make` and `populate` methods of imported tables to automate the importing process.

### make and populate methods

Imported tables come with a special method called `populate`. Let's try calling it:

In [ ]:
# ENTER YOUR CODE! - call the populate method of the table

Notice that the `populate` call produced an error complaining about a method called `make` not being implemented. Let's implement a simple `make` method to elucidate what this is all about:

In [ ]:
@schema
class Neuron(dj.Imported):
    definition = """
    -> Session
    ---
    activity: longblob # electric neuron activity
    """

    def make(self, key):  # The 'make' method takes a single argument called 'key'
        print("Key is", key)

Now let's try to populate the table again:

In [ ]:
# ENTER YOUR CODE! - call the populate method of the table

Calling the `populate` method on a imported table triggers DataJoint to look up all tables that the imported table depends on (i.e. its "parent" tables).

DataJoint will call the `make` function for each **unique combination of entries in the "parent" tables** passing in the primary key of the parent(s).

Because the `Neuron` table depends on the `Session` table, `Neuron`'s `make` method was called for each entry in `Session` with the respective primary key.

In [ ]:
Session()

Note that the `make` method only received the primary key attributes of the `Session` table and not the other non-primary ones.

### Implementing the make method

Now that we have a better understanding of how the `make` method works, let's implement it properly so that it can import the data from the files:

In [ ]:
@schema
class Neuron(dj.Imported):
    definition = """
    -> Session
    ---
    activity: longblob # electric neuron activity
    """

    def make(self, key):
        # Create data file path from key
        data_filepath = "../data/data_{mouse_id}_{session_date}.npy".format(**key)

        # Load data from created file path
        data = np.load(data_filepath)

        # Add the loaded data to the key in the "activity" column
        key["activity"] = data

        # Insert the key into self
        self.insert1(key)

        print("Populated a neuron for mouse_id={mouse_id} and session_date={session_date}".format(**key))

Notice that we add the loaded data to the key dictionary under the `"activity"` column. Afterwards we simply insert the key into `self` (i.e. into the `Neuron` table) because now the key contains all the information needed for a valid entry in the `Neuron` table. The whole job of the `make` method is to create a new entry based on the key it received and insert the new entry into itself.

Finally let's call `populate` once more to populate the `Neuron` table with entries, filling it with data loaded from the data files:

In [ ]:
Neuron.populate()

In [ ]:
Neuron()

What happens if we call `populate` again?

In [ ]:
Neuron.populate()

That's right - nothing! This makes sense because we already populated the `Neuron` table with all the sessions currently in our `Session` table. There is currently nothing more to import.

Let's change that by adding a new session to the `Session` table:

In [ ]:
Session.insert1(
    {
        "mouse_id": 100,
        "session_date": "2017-06-01",
        "experiment_setup": 1,
        "experimenter": "Jacob Reimer",
        "start": "09:52:14",
        "end": "11:19:53",
    }
)

We can list all entries in the `Session` table that have no corresponding entry in the `Neuron` table by using the **exclusion operator** `-`:

In [ ]:
Session - Neuron

Let's see what happens now that we have a new entry in the `Session` table when we populate the `Neuron` table again:

In [ ]:
Neuron.populate()

In [ ]:
Neuron()

As expected the call to `populate` triggered the `make` method of the `Neuron` table which created a new entry corresponding to the newly added session.

## Computations in the data pipeline

Now that we have imported all our data into the pipeline we can start analyzing it.

When performing computations in the DataJoint pipeline we want to think about **what** we want to compute and not how we want to compute it. Once we have a good idea about the "what" we can design tables representing it.

Let's say the "what" in our current pipeline are **activity statistics**, i.e. the mean, standard deviation and maximum of our previously imported neuronal activitiy traces. 

Therefore we need a new table representing the activity statistics of a neuron. Let's start designing that table paying special attention to its dependencies. 

### Neuron activity statistics

Before we create the table to store the computed statistics, let's think about how we might go about computing the statistics for a single neuron.

Let's start by fetching one neuron to work with:

In [ ]:
keys = Neuron().fetch("KEY")
key = keys[0]

In [ ]:
Neuron() & key

First let's grab the activity data stored as a NumPy array. From the last session we know that we can simply fetch it:

In [ ]:
activity = (Neuron & key).fetch("activity")
activity

Note that the `fetch` method wraps the fetched data in a NumPy array. This is the case even if the data itself is a NumPy array to begin with. So here we actually got a NumPy array of a NumPy array. That said we can simply index into the outer array and retrieve our activity data:

In [ ]:
activity[0]

Of course we can get around this nested array issue if we simply use the `insert1` instead of the `insert` method:

In [ ]:
activity = (Neuron & key).fetch1("activity")

In [ ]:
activity

Now we can compute some statistics:

In [ ]:
# ENTER YOUR CODE! - compute the mean activity

In [ ]:
# ENTER YOUR CODE! - compute the standard deviation of the activity

In [ ]:
# ENTER YOUR CODE! - compute the maximum activity

This gives us a good idea on how to:

1. Fetch the activity data of a neuron based on its primary key
2. Compute statistics on the fetched data

Armed with this knowledge let's go ahead and define the `ActivityStatistics` table!

### Defining the ActivityStatistics table

Let's start by working out the definition of the table:

In [ ]:
@schema
class ActivityStatistics(dj.Computed):
    definition = """
    -> Neuron
    ---
    mean: float  # mean activity
    stdev: float  # standard deviation of activity
    max: float  # maximum activity
    """

Did you notice that we are now inheriting from `dj.Computed`? The Computed data tier is the third major data tier in DataJoint. Tables in this data tier compute their **entries based on entries in other tables**. Computed tables are represented as red circles in the ERD:

In [ ]:
dj.Diagram(schema)

Just like imported tables computed tables also make use of the `populate` and `make` methods to create their entries. Let's go ahead and implement the `make` method for our `ActivityStatistics` table:

In [ ]:
# ENTER YOUR CODE! - implement the `make` method

@schema
class ActivityStatistics(dj.Computed):
    definition = """
    -> Neuron
    ---
    mean: float  # mean activity
    stdev: float  # standard deviation of activity
    max: float  # maximum activity
    """

    def make(self, key):
        # 1. Fetch activity
        
        # 2. Compute mean, standard deviation and maximum
        
        # 3. Insert computed values
        
        # 4. Print message indicating for which mouse/session combination statistics were computed

Now we can populate the table:

In [ ]:
ActivityStatistics().populate()

In [ ]:
ActivityStatistics()

Voila! You have computed statistics for the activity of each neuron!

## Spike detection

Although raw neuronal activity traces can be quite interesting, nothing is as exciting as spikes! Let's try to figure out how we could detect spikes from the traces by first visualizing them. As is tradition we start by fetching the primary keys of all our neurons:

In [ ]:
keys = Neuron.fetch("KEY")

Next we fetch the neuronal activity traces:

In [ ]:
activities = (Neuron & keys).fetch("activity")

Lastly we visualize the fetched traces with Matplotlib:

In [ ]:
fig, axes = plt.subplots(1, len(activities), figsize=(16, 4))
for activity, ax in zip(activities, axes.ravel()):
    ax.plot(activity)
    ax.set_xlabel("Time")
    ax.set_ylabel("Activity")

fig.tight_layout()

Nice but let's focus on one trace instead for now so that we can get a better view:

In [ ]:
key = keys[0]

In [ ]:
activity = (Neuron & key).fetch1("activity")

In [ ]:
plt.plot(activity)
plt.xlabel("Time")
plt.ylabel("Activity")
plt.xlim(0, 300);

Looking at this we might be able to use a threshold to detect the spikes. A threshold of 0.5 may be a good starting value:

In [ ]:
threshold = 0.5

above_threshold = (activity > threshold).astype(int)

plt.plot(activity)
plt.plot(above_threshold)
plt.xlabel("Time")
plt.xlim(0, 300);

We want to find the time points at which the activity crossed the threshold, i.e. find the transitions between adjacent time bins where `above_threshold` goes from 0 (`False`) to 1 (`True`):

In [ ]:
rising = (np.diff(above_threshold) > 0).astype(int)  # Find rising edge of crossing the threshold
spikes = np.hstack((0, rising))  # Prepend 0 to account for shortening due to np.diff

plt.plot(activity)
plt.plot(above_threshold)
plt.plot(np.where(spikes > 0), 1, "ro")
plt.xlabel("Time")
plt.xlim(0, 300);

Finally let's also compute the spike count:

In [ ]:
count = spikes.sum()
count

Here is our complete spike detection algorithm:

In [ ]:
# ENTER YOUR CODE! - try different values of threshold!

threshold =     # Enter different threshold values here

above_threshold = (activity > threshold).astype(int)

rising = (np.diff(above_threshold) > 0).astype(int)
spikes = np.hstack((0, rising))

count = spikes.sum()

plt.plot(activity)
plt.plot(above_threshold)
plt.plot(np.where(spikes > 0), 1, "ro")
plt.xlabel("Time")
plt.title(f"Total spike count: {count}")

Notice how the exact spikes detected by the algorithm varies with the chosen `threshold` value. Therefore we might want to try different `threshold` values to see what works best. This means that the `threshold` value ought to be a parameter of the algorithm that we should be able to vary easily.

In other words we want to be able to detect spikes for a **combination** of `Neuron` and `threshold`. Therefore we ought to store the different `threshold` values (and potentially any other parameters) in a separate `Lookup` table.

## Parameter Lookup table

Let's define the `SpikeDetectionParameters` table to hold sets of parameters for our spike detection algorithm. The Python class corresponding to this new table will inherit from `dj.Lookup`. As you might have already guessed Lookup is another data tier in DataJoint. Tables in this tier work similar to tables in the Manual data tier but Lookup tables are generally expected to contain small bits of information that is not specific to a particular experiment and fairly persistent. Our `threshold` parameter certainly falls within that description.

Here we go:

In [ ]:
@schema
class SpikeDetectionParameters(dj.Lookup):
    definition = """
    sdp_id: int  # unique id for spike detection parameter set
    ---
    threshold: float  # threshold for spike detection
    """

Lookup tables are depicted by gray boxes in the ERD:

In [ ]:
dj.Diagram(schema)

## Defining the Spikes table

Now we can define the `Spikes` table in which each entry will be a set of spikes from a particular neuron detected with a particular set of spike detection parameters. In other words each entry will be determined by a **combination of a neuron and a set of spike detection parameters**.

Therefore our `Spikes` table ought to depend on the `Neuron` and `SpikeDetectionParameters` tables. Furthermore we want to store the detected spikes and the spike count in each entry.

The table definition looks like this:

In [ ]:
@schema
class Spikes(dj.Computed):
    definition = """
    -> Neuron
    -> SpikeDetectionParameters
    ---
    spikes: longblob  # detected spikes
    count: int  # total number of detected spikes
    """

In the ERD we can see that our newly defined `Spikes` table depends on both the `Neuron` table and the `SpikeDetectionParameters` table:

In [ ]:
dj.Diagram(schema)

We still need to implement a `make` method for our `Spikes` table that detects the spikes and inserts them into the table. Let's do that now:

In [ ]:
@schema
class Spikes(dj.Computed):
    definition = """
    -> Neuron
    -> SpikeDetectionParameter
    ---
    spikes: longblob  # detected spikes
    count: int  # total number of detected spikes
    """

    def make(self, key):
        print("Populating for: ", key)

        activity = (Neuron & key).fetch1("activity")
        threshold = (SpikeDetectionParameters & key).fetch1("threshold")

        above_threshold = (activity > threshold).astype(int)
        rising = (np.diff(above_threshold) > 0).astype(int)
        spikes = np.hstack((0, rising))

        count = spikes.sum()
        print(f"Detected {count} spikes!\n")

        key["spikes"] = spikes
        key["count"] = count
        self.insert1(key)

The implementation of the `make` method is basically identical to our earlier spike detection algorithm except that we now fetch the value of the `threshold` parameter from our `SpikeDetectionParameters` table.

As expected the `Spikes` table inherits the primary key attributes from both `Neuron` (`mouse_id`, `session_date`) and `SpikeDetectionParameters` (`sdp_id`):

In [ ]:
Spikes()

### Populating the Spikes table

We are now ready to populate the `Spikes` table. DataJoint will call its `make` method for each valid combination of the parent tables - `Neuron` and `SpikeDetectionParameters`:

In [ ]:
# ENTER YOUR CODE - populate the Spikes table

Hm, weird. Nothing happened! Why might that be the case?

Looking at `SpikeDetectionParameters` reveals the issue:

In [ ]:
SpikeDetectionParameters()

That is right! We still need to add a parameter set to this table. Let's do that now:

In [ ]:
SpikeDetectionParameters.insert1((0, 0.5))

In [ ]:
SpikeDetectionParameters()

Nothing can go wrong now...

In [ ]:
# ENTER YOUR CODE! - populate the Spikes table for real

In [ ]:
Spikes()

Finally! We are now detecting spikes using DataJoint!

## Trying out other parameter values

Let's try a different `threshold` value to see how that affects the result:

In [ ]:
SpikeDetectionParameters.insert1((1, 0.9))

In [ ]:
SpikeDetectionParameters()

In [ ]:
# ENTER YOUR CODE! - populate the missing entries in the Spikes table

In [ ]:
Spikes()

As you can see spikes detected with different parameter sets can live happily next to each other without any confusion as to what is what.

## Deleting entries "upstream"

Now let's say for some reason you decide that you want to get rid of all the spikes detected with a `threshold` value of 0.5. You could do this by restricting the `Spikes` table down to the entries that depend on the specific parameter ID (i.e. `sdp_id = 0`) and delete them:

In [ ]:
(Spikes & "sdp_id = 0").delete()

Although this would delete the unwanted spikes it would not delete the offending parameter set in the `SpikeDetectionParameters` table. So a much easier way is to just delete the parameter set and let DataJoint cascade the deletion down to the `Spikes` table:

In [ ]:
SpikeDetectionParameters & "sdp_id = 0"

In [ ]:
(SpikeDetectionParameters & "sdp_id = 0").delete()

In [ ]:
Spikes()

## Summary

Congratulations! In this webinar we have extended our DataJoint pipeline with a table to represent recorded neural data (`Neuron` as a Imported table), tables that perform and represent computation results (`ActivityStatistict` and `Spikes` as Computed tables) and a table to hold computation parameters (`SpikeDetectionParameters` as a Lookup table).

Let's take a final look at our current schema:

In [ ]:
dj.Diagram(schema)

Our pipeline is still fairly simple but now it is completely capable of handling analysis!